In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'
import pygame  # it is important to import pygame after that
import time

#Pandas Settings
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def scanner():
    
    url = 'http://18.231.88.179/sites/scanner.php?timeI=0&timeF=0'      
    link = requests.get(url)
    page = BeautifulSoup(link.text, 'html.parser')

    jogo = page.find_all('tr')
    head = page.find_all('',{'class':'text-center'})
    tam_head = len(page.find_all('',{'class':'text-center'}))

    # para dá toque de alerta quando chamada a função de toque
    pygame.init()
    pygame.mixer.music.load('toque.mpeg')
    
    # flag para indicar quando a função deve parar
    func = True
    if (len(jogo) - 1) == 0:
        func = False

    print(f'quant jogos: {len(jogo) - 1}')
            
    # varredura de jogos
    for i in range(1,len(jogo)):
        print(f'jogo {i}')
        
        # tentando mais de uma vez pegar os dados do jogo caso surja erro
        for k in range(3):        
            try:

                # varredura de atributos do jogo
                for j in range(11):

                    # caminho de cada dado mudando apenas o "j" para cada dado
                    dado = jogo[i].find_all('td')[j]

                    # tempo de jogo
                    if j == 0:
                        if (dado.text == 'HT') or (dado.text == 'FT'):
                            tempo = dado.text
                            print(f'tempo {tempo}')
                        else:
                            tempo = dado.text.split("'")[0].split('+')[0]
                            print(f'tempo {tempo}')

                    # placar
                    elif j == 1:
                        placar = dado.text.replace(' - ','x')

                    # equipes
                    elif j == 2:
                        times = dado.text
                        print(times)

                    # chutes no gol
                    elif j == 3: 
                        chutesnogolcasa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(chutesnogolcasa)[1]) == '':
                            chutesnogolvisitante = chutesnogolcasa
                        else:
                            chutesnogolvisitante = dado.text.split(chutesnogolcasa)[1]

                    # chutes fora do gol 
                    elif j == 4:
                        chutesforagolcasa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(chutesforagolcasa)[1]) == '':
                            chutesforagolvisitante = chutesforagolcasa
                        else:
                            chutesforagolvisitante = dado.text.split(chutesforagolcasa)[1]
                            
                    # escanteios
                    elif j == 5:
                        escanteioscasa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(escanteioscasa)[1]) == '':
                            escanteiosvisitante = escanteioscasa
                        else:
                            escanteiosvisitante = dado.text.split(escanteioscasa)[1]

                    # ataques perigosos
                    elif j == 6:
                        ataquesperigososcasa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(ataquesperigososcasa)[1]) == '':
                            ataquesperigososvisitante = ataquesperigososcasa
                        else:
                            ataquesperigososvisitante = dado.text.split(ataquesperigososcasa)[1]

                    # posse de bola
                    elif j == 7:
                        possecasa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(possecasa)[1]) == '':
                            possevisitante = possecasa
                        else:
                            possevisitante = dado.text.split(possecasa)[1]

                    # PI 2
                    elif j == 8:
                        pi2casa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(pi2casa)[1]) == '':
                            pi2visitante = pi2casa
                        else:
                            pi2visitante = dado.text.split(pi2casa)[1]

                    # PI 1
                    elif j == 9:
                        pi1casa = dado.find('div').text

                        # separando atributos de casa e visitante
                        if (dado.text.split(pi1casa)[1]) == '':
                            pi1visitante = pi1casa
                        else:
                            pi1visitante = dado.text.split(pi1casa)[1]

                    # link com odds
                    elif j == 10:
                        linkodds = 'http://18.231.88.179/sites/' + dado.find('a').get('href').split('/')[1]

                        # pegando as odds e informações do campeonato se tiver
                        link = requests.get(linkodds)
                        page2 = BeautifulSoup(link.text, 'html.parser')
                        
                        # Pegando o nome das equipes direto da tabela de odds para comparar se estamos nas odds dos times certos
                        # Não pego direto de "Evento" pq nem sempre os nomes estão disponíveis ali
                        time1 = page2.find_all('tr')[1].find_all('td')[1].text
                        time2 = page2.find_all('tr')[3].find_all('td')[0].text
                        times_secao_odds = time1 + ' x ' + time2 
                        
                        # verificando se a página de odds é a do time em questão
                        if times_secao_odds == times:
                            print(f'link com as odds {linkodds}')
                            
                            # data
                            data = page2.find('h6').text.split('Atualizado em ')[1][0:10]

                            # campeonato
                            campeonato = page2.find_all('center')[0].text.split('Campeonato: ')[1].split('Evento: ')[0]
                            print(campeonato)

                            # odds - match odds
                            
                            # home back e lay
                            home_back = page2.find_all('tr')[1].find('td',{'bgcolor':'#1E90FF'}).text
                            print(home_back)                            
                            home_lay = page2.find_all('tr')[1].find('td',{'bgcolor':'#FF69B4'}).text
                            print(home_lay)
                            
                            # empate back e lay
                            draw_back = page2.find_all('tr')[2].find('td',{'bgcolor':'#1E90FF'}).text
                            print(draw_back)                            
                            draw_lay = page2.find_all('tr')[2].find('td',{'bgcolor':'#FF69B4'}).text
                            print(draw_lay)
                            
                            # away back e lay
                            away_back = page2.find_all('tr')[3].find('td',{'bgcolor':'#1E90FF'}).text
                            print(away_back)                            
                            away_lay = page2.find_all('tr')[3].find('td',{'bgcolor':'#FF69B4'}).text
                            print(away_lay)
                            
                            # odds - over e under
                            
                            # Over 1.5 back e lay
                            o15_back = page2.find_all('tr')[4].find('td',{'bgcolor':'#1E90FF'}).text
                            print(o15_back)                            
                            o15_lay = page2.find_all('tr')[4].find('td',{'bgcolor':'#FF69B4'}).text
                            print(o15_lay)
                            
                            # Under 1.5 back e lay
                            u15_back = page2.find_all('tr')[5].find('td',{'bgcolor':'#1E90FF'}).text
                            print(u15_back)                            
                            u15_lay = page2.find_all('tr')[5].find('td',{'bgcolor':'#FF69B4'}).text
                            print(u15_lay)
                            
                            # Over 2.5 back e lay
                            o25_back = page2.find_all('tr')[6].find('td',{'bgcolor':'#1E90FF'}).text
                            print(o25_back)                            
                            o25_lay = page2.find_all('tr')[6].find('td',{'bgcolor':'#FF69B4'}).text
                            print(o25_lay)
                            
                            # Under 2.5 back e lay
                            u25_back = page2.find_all('tr')[7].find('td',{'bgcolor':'#1E90FF'}).text
                            print(u25_back)                            
                            u25_lay = page2.find_all('tr')[7].find('td',{'bgcolor':'#FF69B4'}).text
                            print(u25_lay)

                            # Over 3.5 back e lay
                            o35_back = page2.find_all('tr')[8].find('td',{'bgcolor':'#1E90FF'}).text
                            print(o35_back)                            
                            o35_lay = page2.find_all('tr')[8].find('td',{'bgcolor':'#FF69B4'}).text
                            print(o35_lay)

                            # Under 3.5 back e lay
                            u35_back = page2.find_all('tr')[9].find('td',{'bgcolor':'#1E90FF'}).text
                            print(u35_back)                            
                            u35_lay = page2.find_all('tr')[9].find('td',{'bgcolor':'#FF69B4'}).text
                            print(u35_lay)
                            
                            # Over 4.5 back e lay
                            o45_back = page2.find_all('tr')[10].find('td',{'bgcolor':'#1E90FF'}).text
                            print(o45_back)                            
                            o45_lay = page2.find_all('tr')[10].find('td',{'bgcolor':'#FF69B4'}).text
                            print(o45_lay)
                            
                            # Under 4.5 back e lay
                            u45_back = page2.find_all('tr')[11].find('td',{'bgcolor':'#1E90FF'}).text
                            print(u45_back)                            
                            u45_lay = page2.find_all('tr')[11].find('td',{'bgcolor':'#FF69B4'}).text
                            print(u45_lay)
                            
                            # odds - correct score
 
                            # Goleada casa
                            goleadacasa_back = page2.find_all('td',{'bgcolor':'#1E90FF'})[27].text
                            print(goleadacasa_back)                            
                            goleadacasa_lay = page2.find_all('td',{'bgcolor':'#FF69B4'})[27].text
                            print(goleadacasa_lay)

                            # Goleada fora
                            goleadafora_back = page2.find_all('td',{'bgcolor':'#1E90FF'})[28].text
                            print(goleadafora_back)                            
                            goleadafora_lay = page2.find_all('td',{'bgcolor':'#FF69B4'})[28].text
                            print(goleadafora_lay)
                            
                            # Outro empate
                            outroempate_back = page2.find_all('td',{'bgcolor':'#1E90FF'})[29].text
                            print(outroempate_back)                            
                            outroempate_lay = page2.find_all('td',{'bgcolor':'#FF69B4'})[29].text
                            print(outroempate_lay)
                            
                            placar_secao_odds = page2.find_all('center')[0].text.split('Placar: ')[1].split(' Tempo: ')[0].replace(' - ', 'x')
                            
                            if placar_secao_odds == placar:
                                # flag para checar se nao houve gol (e portanto mudança de odd) enquanto coletava as odds
                                mudou_placar = False
                                
                            else:
                                # flag para checar se nao houve gol (e portanto mudança de odd) enquanto coletava as odds
                                mudou_placar = True
                            
                            
                print(f'times {times}')
                
                break
                
            except:
                continue
            
            
        # CRIANDO UM DATAFRAME COM OS DADOS DA PARTIDA A CADA 10 MIN E ARMAZENANDO PARA FORMAR A BASE DE DADOS
        if (tempo == 'HT' or tempo == 'FT' or (int(tempo) % 10) == 0 or int(tempo) == 45) and (mudou_placar == False):
            a = '3'
            # ler o dataframe anterior para depois concatenar com o novo
            try:
                anterior_df = pd.read_csv('dados_soccer_stats.csv')
            except:
                anterior_df = pd.DataFrame()
                pass

            # criando dataframe novo a cada 10 min
            atual_df = pd.DataFrame(                
                {'data':[data], 'campeonato':[campeonato] ,'times':[times], 'tempo':[tempo], 'placar':[placar], 'chutesnogolgolcasa':[chutesnogolcasa],
                 'chutesnogolvisitante':[chutesnogolvisitante], 'chutesforagolcasa':[chutesforagolcasa],
                 'chutesforagolvisitante':[chutesforagolvisitante], 'escanteioscasa':[escanteioscasa],
                 'escanteiosvisitante':[escanteiosvisitante], 'ataquesperigososcasa':[ataquesperigososcasa],
                 'ataquesperigososvisitante':[ataquesperigososvisitante], 'possecasa':[possecasa], 'possevisitante':[possevisitante],
                 'pi2casa':[pi2casa], 'pi2visitante':[pi2visitante], 'pi1casa':[pi1casa], 'pi1visitante':[pi1visitante],
                'home_back':[home_back],'home_lay':[home_lay],'away_back':[away_back],'away_lay':[away_lay],'draw_back':[draw_back],
                'draw_lay':[draw_lay],'o15_back':[o15_back],'o15_lay':[o15_lay],'u15_back':[u15_back],'u15_lay':[u15_lay],'o25_back':[o25_back],
                'o25_lay':[o25_lay],'u25_back':[u25_back],'u25_lay':[u25_lay],'o35_back':[o35_back],'o35_lay':[o35_lay],'u35_back':[u35_back],
                'u35_lay':[u35_lay],'o45_back':[o45_back],'o45_lay':[o45_lay],'u45_back':[u45_back],'u45_lay':[u45_lay],
                'goleadacasa_back':[goleadacasa_back],'goleadacasa_lay':[goleadacasa_lay],'goleadafora_back':[goleadafora_back],
                'goleadafora_lay':[goleadafora_lay],'outroempate_back':[outroempate_back],'outroempate_lay':[outroempate_lay]}
            ) 
           
            # colocando o dataframe anterior e o atual em uma lista
            list_frames = [anterior_df, atual_df]

            # concatenando a lista de dataframe em um único dataframe
            final_df = pd.concat(list_frames, sort=False)

            try:
                # eliminando coluna indesejada                
                final_df.drop(['Unnamed: 0'], inplace=True, axis=1)
                
                # atualizando o indice
                final_df = final_df.reset_index(drop=True)
                
                # convertendo valores possiveis no dataframe em numericos. Necessario para a busca de dados duplicados
                final_df = final_df.apply(pd.to_numeric, errors='ignore')
                
                # filtrando o tempo em questão e os times em questão para ver se já foram armazenados (verificando se estão duplicados)
                cont_dados_duplicados = final_df.duplicated(subset=['tempo', 'times'], keep='first').sum()

                if cont_dados_duplicados == 0 :

                    # salvando o dataframe construído a cada 10 min de cada partida
                    final_df.to_csv('dados_soccer_stats.csv')
                    print('salvei a partir do try')

                    
            except:
                # atualizando o indice
                final_df = final_df.reset_index(drop=True)
                
                # convertendo valores possiveis no dataframe em numericos. Necessario para a busca de dados duplicados
                final_df = final_df.apply(pd.to_numeric, errors='ignore')
                
                # salvando o dataframe construído a cada 10 min de cada partida
                final_df.to_csv('dados_soccer_stats.csv')
                print('salvei a partir do except')
                pass
            
            
            
        # aviso sonoro 

        # ambas marcam
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 3) and (placar == '0x1' or placar == '0x2' or placar == '0x3') and (int(tempo) <= 45):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

            elif (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '3x0') and (int(tempo) <= 45):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

        else:
            if (int(chutesnogolcasa) >= 3) and (placar == '0x1' or placar == '0x2' or placar == '0x3') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

            elif (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '3x0') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)



        # over gols
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 3) and (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '0x1' or placar == '0x2') and (int(tempo) <= 45):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

            elif (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) >= 2) and (int(tempo) <= 20):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

        else:
            if (int(chutesnogolcasa) >= 3) and (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '0x1' or placar == '0x2') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)




        # under gols
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) == 0) and (int(tempo) >= 35 and int(tempo) <= 45) and (placar == '1x0' or placar == '2x0'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

            elif (int(chutesnogolvisitante) >= 2) and (int(chutesnogolcasa) == 0) and (int(tempo) >= 35 and int(tempo) <= 45) and (placar == '0x1' or placar == '0x2'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

        else:
            if (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) == 0) and (placar == '1x0' or placar == '2x0') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)

            elif (int(chutesnogolvisitante) >= 2) and (int(chutesnogolcasa) == 0) and (placar == '0x1' or placar == '0x2') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(2)


        print('\n\n')
    return func

In [ ]:
# chamando a função para fazer scanner in live
func = True
count = 0
while (func == True or count < 3):
    try:
        func = scanner()
        if func == False:
            count += 1
            time.sleep(0.5)
        else:
            count = 0
            time.sleep(1)
    except:
        time.sleep(0.5)
        continue
print('Nenhum jogo ao vivo neste momento.')

quant jogos: 18
jogo 1
tempo 39
Arsenal x Chelsea
link com as odds http://18.231.88.179/sites/odds.php?id=0
English Premier League - GB
1.65
1.66
4.2
4.3
6.4
6.6
1.13
1.14
8.4
8.6
1.66
1.67
2.48
2.52
3.2
3.3
1.44
1.46
7.6
7.8
1.14
1.15
16
17
50
65
530
590
times Arsenal x Chelsea



jogo 2
tempo 50
Alanyaspor x Konyaspor
link com as odds http://18.231.88.179/sites/odds.php?id=1
Turkish Super League - TR
1.87
1.89
2.72
2.78
9.4
9.8




1.36
1.39
3.6
3.75
2.74
2.82
1.55
1.58
7.4
7.8
1.15
1.16
13.5
15
260
370
1000

times Alanyaspor x Konyaspor



jogo 3
tempo 38
Hearts x Aberdeen
link com as odds http://18.231.88.179/sites/odds.php?id=2
Scottish Premiership - GB
2.74
2.8
2.5
2.56
4.1
4.3
2.44
2.46
1.68
1.69
5.9
6.4
1.2
1.21
19.5
21
1.05
1.06
65
110
1.01
1.02
100
160
24
410
1000

times Hearts x Aberdeen



jogo 4
tempo 79
Gillingham x Rochdale
link com as odds http://18.231.88.179/sites/odds.php?id=3
English League 1 - GB
3.05
3.1
1.66
1.68
12.5
13
11
12
1.09
1.1
100


1.01
230


1.01
1000


link com as odds http://18.231.88.179/sites/odds.php?id=14
Turkish 1 Lig - TR
28
30
9.4
10
1.15
1.17




1.22
1.23
5.5
5.6
1.98
2.02
1.98
2.02
4.2
4.5
1.28
1.31
14.5
130
5.2
6
460

times Istanbulspor x Adana Demirspor



jogo 16
tempo 75
Esae FC x Zanaco FC
link com as odds http://18.231.88.179/sites/odds.php?id=15
CAF Confederation Cup - 
8.2
8.8
1.77
1.84
2.98
3.2
7.2
8.6
1.13
1.16
42
110
1.01
1.02
190


1.01
770


1.01
150

150

12.5

times Esae FC x Zanaco FC



jogo 17
tempo 8
CF Calamocha x Utebo
link com as odds http://18.231.88.179/sites/odds.php?id=16
Spanish Tercera Division - ES
2.52
2.62
3.15
3.3
3.3
3.5
1.51
1.55
2.82
2.96
2.56
2.58
1.64
1.65
5.4
5.9
1.2
1.23
13
15.5
1.07
1.09
27
40
48
65
220

times CF Calamocha x Utebo



jogo 18
tempo 39
TAS Casablanca x RAC Casablanca
link com as odds http://18.231.88.179/sites/odds.php?id=17
Moroccan Botola 2 - MA
1.75
1.94
3.25
3.6
5.7
6.6
1.33
1.37
3.75
4
2.6
2.8
1.56
1.63
6.8
8.2
1.14
1.18
17.5
50
1.03
1.06
28
80
65

7.8

times TAS 

link com as odds http://18.231.88.179/sites/odds.php?id=10
Italian Serie B - IT
1.39
1.41
4.7
4.9
12.5
13
1.24
1.26
4.9
5.1
2.12
2.16
1.87
1.9
4.6
4.9
1.27
1.28
13
14
1.07
1.09
15.5
19
170

1000

times Spezia x Salernitana



jogo 12
tempo 39
Mumbai City FC x Hyderabad FC
link com as odds http://18.231.88.179/sites/odds.php?id=11
Indian Super League - IN
1.32
1.34
5.3
5.7
15
16
1.14
1.15
7.8
8.2
1.63
1.66
2.52
2.6
3.1
3.25
1.45
1.48
6.8
7.4
1.15
1.18
7
7.4
160

120

times Mumbai City FC x Hyderabad FC
GRANDE CHANCE PARA UNDER
Mumbai City FC x Hyderabad FC



jogo 13
tempo 71
Nasr EL Fedjoudj x US Biskra
link com as odds http://18.231.88.179/sites/odds.php?id=12
Algerian Cup - DZ
4.9
5
2
2.04
3.25
3.4




1.74
1.83
2.2
2.34
5.1
5.9
1.2
1.25
21
36
1.03
1.05
42

26

230

times Nasr EL Fedjoudj x US Biskra



jogo 14
tempo 77
AS Slimane x ES Metlaoui
link com as odds http://18.231.88.179/sites/odds.php?id=13
Tunisian Ligue Professionelle 1 - TN
1.11
1.12
9.6
10
160
180
2.26
2.42
1.72
1.79
